In [1]:
import scipy.stats as stats
import pandas as pd
import psycopg2
import numpy as np
import math
import json
import os
import backtrader as bt
import datetime
import matplotlib

In [2]:
# Connect to db
conn = psycopg2.connect('dbname=securities user=postgres')
cur = conn.cursor()

In [57]:
# Get all price data
query_all = '''
    SELECT *
    FROM prices
    ORDER BY symbol ASC, datetime ASC
'''
cur.execute(query_all)
data = cur.fetchall()
price_df = pd.DataFrame(data, columns = ['symbol', 'date', 'open', 'high', 'low', 'close'])
price_df.loc[:, 'date'] = pd.to_datetime(price_df['date'])
price_df = price_df.set_index('date')

# Get list of all unique tickers
ticker_list = price_df.symbol.unique()

In [109]:
class SMAIndicator(bt.Indicator):
    lines = ('sma',)
    params = {
        'long_ma': None,
        'short_ma': None,
    }
    
    def __init__(self):
        self.addminperiod(max(self.params.long_ma, self.params.short_ma))
        self.long_period = max(self.params.long_ma, self.params.short_ma)
        self.short_period = min(self.params.long_ma, self.params.short_ma)
    
    def next(self):
        sma_long = np.mean(self.data.get(size = self.long_period))
        sma_short = np.mean(self.data.get(size = self.short_period))
        self.lines.sma[0] = sma_short - sma_long

In [139]:
class TestStrategy(bt.Strategy):
    
    params = {
        'trade_freq': None,
        'feed_name': None,
    }
    
    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
        
    def __init__(self):
        self.dataclose = self.datas[0].close
        self.order = None
        self.sma = SMAIndicator(long_ma = 30, short_ma = 10)
        self.days = 0
        
        self.inds = dict()
        for i, d in enumerate(self.datas):
            self.inds[d] = dict()
            self.inds[d]['sma'] = self.sma
        
        
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return
        
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, PRICE: %.2f, Cost: %.2f, Comm %.2f' % (
                        order.executed.price,
                        order.executed.value,
                        order.executed.comm,
                    )
                )
            else:
                self.log(
                    'SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' % (
                        order.executed.price,
                        order.executed.value,
                        order.executed.comm,
                    )
                )
            self.bar_executed = len(self)
            
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None
        
    def notify_trade(self, trade):
        if not trade.isclosed:
            return
        
        self.log(
            'OPERATION PROFIT, GROSS %.2f, NET %.2f' % (
                trade.pnl,
                trade.pnlcomm,
            )
        )
        
    def next(self):
        self.days += 1
        
        if self.days > self.params.trade_freq - 1:
            self.days = 0
            for i, d in enumerate(self.datas):
                dt, dn = self.datetime.date(), d._name
                if not self.order:
                    if self.inds[d]['sma'][0] > 0:
                        self.log('BUY CREATE, %.2f' % d.close[0])
                        self.order = self.buy(data = d, size = 2)
                    else:
                        self.log('SELL CREATE, %.2f' % d.close[0])
                        self.order = self.sell(data = d, size = 2)
                print(self.getposition(d))
        
        '''
        if self.days > self.params.trade_freq - 1:
            self.days = 0
            if not self.position:
                if self.sma > 0:
                    self.log('BUY CREATE, %.2f' % self.dataclose[0])
                    self.order = self.buy()
            elif self.sma < 0:
                self.log('SELL CREATE, %.2f' % self.dataclose[0])
                self.order = self.sell()
        '''

In [140]:
cerebro = bt.Cerebro()

# Load all feeds into Cerebro
for ticker in ['SPY', 'IWO']:
    ticker_df = price_df[price_df['symbol'] == ticker]
    cerebro.adddata(bt.feeds.PandasData(dataname = ticker_df, datetime = -1))

cerebro.addstrategy(TestStrategy, trade_freq = 10, feed_name = ticker_list)

0

In [141]:
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
cerebro.run()
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 10000.00
2019-10-22, BUY CREATE, 299.01
--- Position Begin
- Size: 0
- Price: 0.0
- Price orig: 0.0
- Closed: 0
- Opened: 0
- Adjbase: None
--- Position End
--- Position Begin
- Size: 0
- Price: 0.0
- Price orig: 0.0
- Closed: 0
- Opened: 0
- Adjbase: None
--- Position End
2019-10-23, BUY EXECUTED, PRICE: 298.73, Cost: 597.46, Comm 0.00
2019-11-05, BUY CREATE, 307.03
--- Position Begin
- Size: 2
- Price: 298.73
- Price orig: 0.0
- Closed: 0
- Opened: 2
- Adjbase: 307.03
--- Position End
--- Position Begin
- Size: 0
- Price: 0.0
- Price orig: 0.0
- Closed: 0
- Opened: 0
- Adjbase: None
--- Position End
2019-11-06, BUY EXECUTED, PRICE: 307.03, Cost: 614.06, Comm 0.00
2019-11-19, BUY CREATE, 311.93
--- Position Begin
- Size: 4
- Price: 302.88
- Price orig: 298.73
- Closed: 0
- Opened: 2
- Adjbase: 311.93
--- Position End
--- Position Begin
- Size: 0
- Price: 0.0
- Price orig: 0.0
- Closed: 0
- Opened: 0
- Adjbase: None
--- Position End
2019-11-20, BUY EXECUTED, P

In [52]:
price_df[price_df['symbol'] == 'VTV'].head()

,symbol,date,open,high,low,close
52545,VTV,2019-08-28,105.95,107.1626,105.8103,107.11
52546,VTV,2019-08-29,108.03,108.4800,107.6000,108.24
52547,VTV,2019-08-30,108.86,109.0800,108.2598,108.52
52548,VTV,2019-09-03,107.87,108.2000,107.2800,108.12
52549,VTV,2019-09-04,108.97,109.2300,108.7500,109.17


In [53]:
price_df.symbol.unique()

array(['AAXJ', 'ACWI', 'ACWV', 'AGG', 'ASHR', 'ASHS', 'BBEU', 'BBJP',
       'BBSA', 'BBUS', 'BIL', 'BKLN', 'BLV', 'BND', 'BNDW', 'BNDX',
       'BSCP', 'BSCR', 'BSCS', 'BSV', 'BWX', 'BWZ', 'CNYA', 'CORP',
       'CRBN', 'DEM', 'DES', 'DON', 'DSI', 'DVY', 'DWX', 'DXJS', 'EBND',
       'EDV', 'EEM', 'EEMV', 'EFA', 'EFV', 'EMB', 'EMLC', 'EQAL', 'EWJ',
       'EZU', 'FBND', 'FCOR', 'FDLO', 'FLOT', 'FLTB', 'FNDA', 'FNDB',
       'FNDC', 'FNDF', 'FNDX', 'FTC', 'GEM', 'GMF', 'GOVT', 'GVAL', 'GVI',
       'GWX', 'HYD', 'HYG', 'HYLD', 'IAF', 'IAGG', 'ICSH', 'IEF', 'IEFA',
       'IEI', 'IEMG', 'IEUR', 'IEV', 'IGOV', 'IGSB', 'IJH', 'IJJ', 'IJK',
       'IJR', 'IJS', 'IJT', 'ILTB', 'ISHG', 'ISTB', 'ITOT', 'IUSB',
       'IUSV', 'IVE', 'IVOG', 'IVOO', 'IVV', 'IWB', 'IWC', 'IWD', 'IWM',
       'IWN', 'IWO', 'IWP', 'IWR', 'IWS', 'IWV', 'IXUS', 'IYY', 'JDIV',
       'JKH', 'JMIN', 'JMOM', 'JMST', 'JNK', 'JPEM', 'JPGB', 'JPHY',
       'JPIN', 'JPMB', 'JPME', 'JPSE', 'JPST', 'JPUS', 'JQUA', 'JVAL',
  